In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage import io
import os
import gc
import pickle

from skimage.filters import gaussian
from skimage.morphology import remove_small_objects
from scipy.ndimage import center_of_mass
from stardist.models import StarDist2D
from csbdeep.utils import normalize
from skimage.segmentation import clear_border
import cv2

In [4]:
channelNames=['Tau','Abeta','MAP2','LMNB','DAPI']
dataDir='/data/xzhang/neuro_new/newdata/tauAbeta_tiff'
segsavepath=os.path.join(dataDir,'segmentations')
if not os.path.exists(segsavepath):
    os.mkdir(segsavepath)
savedir_processed=os.path.join(dataDir,'processed')
if not os.path.exists(savedir_processed):
    os.mkdir(savedir_processed)

In [4]:
model_stardist = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [17]:
scaleFactor=0.1626/0.2031

In [ ]:

for path in os.listdir(dataDir):
    print(path)
    if not os.path.exists(os.path.join(segsavepath,path)):
        os.mkdir(os.path.join(segsavepath,path))
    for fname in os.listdir(os.path.join(dataDir,path)):
        if '.npy' not in fname:
            continue
        pID=fname[:-4]
        print(pID)
        if os.path.exists(os.path.join(segsavepath,path,pID+'.tif')):
            print('skip segmented')
            continue
        img2d=np.load(os.path.join(dataDir,path,pID+'.npy'))[4]
        print(img2d.shape)
        labeled_nuclei, linfo = model_stardist.predict_instances(gaussian(normalize(img2d),5/scaleFactor))
        io.imsave(os.path.join(segsavepath,path,pID+'.tif'),labeled_nuclei)

In [ ]:
allCoord=None
allImgNames=None
allPatientID=None
allCat=None
allHC_staci=None
allHC_std=None

imgSize_target=128
imgSize=imgSize_target*scaleFactor
radius=int(imgSize/2)
minNucSize=300
laminThreshold=0.3 #by area
minmaxScale=[(10,99.99),(90,99.99),(10,99.99),(10,99.99),(10,99.99)]
abetaDist=[400,800]

allAbeta_mean={400:[],800:[]}
allAbeta_area={400:[],800:[]}

for path in os.listdir(dataDir):
    print(path)
    for fname in os.listdir(os.path.join(dataDir,path)):
        if '.npy' not in fname:
            continue
        pID=fname[:-4]
        print(pID)
        img2d=np.load(os.path.join(dataDir,path,pID+'.npy')).astype(float)
        imgSeg=io.imread(os.path.join(segsavepath,path,pID+'.tif'))
        
        #scale intensity
        for c in range(img2d.shape[0]):
#             if c!=4:
#                 continue
            img2d[c]=np.clip(img2d[c],np.percentile(img2d,minmaxScale[c][0]),np.percentile(img2d,minmaxScale[c][1]))
            img2d[c]=(img2d[c]-np.min(img2d[c]))/(np.max(img2d[c])-np.min(img2d[c]))
        
        #HC of cells
        currCoord=np.zeros((np.unique(imgSeg).size-1,2))
        currHC_staci=np.zeros(np.unique(imgSeg).size-1)
        currHC_std=np.zeros(np.unique(imgSeg).size-1)
        imgCount=0
        for idx in np.unique(imgSeg):
            if idx==0:
                continue
            current_nuc=imgSeg==idx
            
            x,y = center_of_mass(current_nuc)
            x=int(x)
            y=int(y)
            if x-radius<0 or x+radius>(img2d.shape[1]) or y-radius<0 or y+radius>(img2d.shape[2]):
                continue
            
            if np.sum(current_nuc)<minNucSize:
                lmnbPix=img2d[3][current_nuc]>0
                if np.sum(lmnbPix)/np.sum(current_nuc)<laminThreshold:
                    continue
            
            currImgs_0=np.copy(img2d[4,x-radius:x+radius,y-radius:y+radius])
            currImgs_0_seg=imgSeg[x-radius:x+radius,y-radius:y+radius]
            currImgs_0[currImgs_0_seg!=idx]=0
            currHC_staci_thresh= (0.4*np.max(currImgs_0) + np.min(currImgs_0[currImgs_0_seg==idx]) + 0.35*(np.max(currImgs_0)-np.min(currImgs_0[currImgs_0_seg==idx])))/2
            currHC_std_thresh=np.mean(currImgs_0)+1.5*np.std(currImgs_0)
            currHC_staci[imgCount]=np.sum(currImgs_0>currHC_staci_thresh)/np.sum(currImgs_0>0)
            currHC_std[imgCount]=np.sum(currImgs_0>currHC_std_thresh)/np.sum(currImgs_0>0)
            currCoord[imgCount]=np.array([x,y])
            
            imgCount+=1
        currHC_staci=currHC_staci[:imgCount]
        currHC_std=currHC_std[:imgCount]
        currCoord=currCoord[:imgCount]
        currImgName=np.repeat(pID,imgCount)
        currPatientID=np.repeat(pID.split('-')[0]+'-'+pID.split('-')[1],imgCount)
        currCat=np.repeat(path,imgCount)
        if allCoord is None:
            allImgNames=currImgName
            allCoord=currCoord
            allPatientID=currPatientID
            allCat=currCat
            allHC_staci=currHC_staci
            allHC_std=currHC_std
        else:
            allImgNames=np.concatenate((allImgNames,currImgName))
            allCoord=np.concatenate((allCoord,currCoord),axis=0)
            allPatientID=np.concatenate((allPatientID,currPatientID))
            allCat=np.concatenate((allCat,currCat))
            allHC_staci=np.concatenate((allHC_staci,currHC_staci))
            allHC_std=np.concatenate((allHC_std,currHC_std))
        
        #abeta - threshold; mean intensity within a distance
        # threshold -> abeta pixel counts within a distance
        for d in abetaDist:
            print('dist', d)
            currABmean=np.zeros(imgCount)-1
            currABarea=np.zeros(imgCount)-1
            for idx in range(imgCount):
                x,y=currCoord[idx].astype(int)
                radiusAB=int(d/2)
                if x-radiusAB<0 or x+radiusAB>(img2d.shape[1]) or y-radiusAB<0 or y+radiusAB>(img2d.shape[2]):
                    continue
                currImgs_ab=img2d[1,x-radiusAB:x+radiusAB,y-radiusAB:y+radiusAB]
                currABmean[idx]=np.mean(currImgs_ab)
                currABarea[idx]=np.sum(currImgs_ab>0)/currImgs_ab.size
            allAbeta_area[d]=np.concatenate((allAbeta_area[d],currABarea))
            allAbeta_mean[d]=np.concatenate((allAbeta_mean[d],currABmean))
            

with open(os.path.join(savedir_processed,'allImgNames'), 'wb') as output:
    pickle.dump(allImgNames, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allCoord'), 'wb') as output:
    pickle.dump(allCoord, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allPatientID'), 'wb') as output:
    pickle.dump(allPatientID, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allHC_staci'), 'wb') as output:
    pickle.dump(allHC_staci, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allHC_std'), 'wb') as output:
    pickle.dump(allHC_std, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allAbeata_mean'), 'wb') as output:
    pickle.dump(allAbeta_mean, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allAbeata_area'), 'wb') as output:
    pickle.dump(allAbeta_area, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allCat'), 'wb') as output:
    pickle.dump(allCat, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
allCoord=None
allImgNames=None
allPatientID=None
allCat=None
allHC_staci=None
allHC_std=None

imgSize_target=128
imgSize=imgSize_target*scaleFactor
radius=int(imgSize/2)
minNucSize=300
laminThreshold=0.3 #by area
minmaxScale=[(10,99.99),(90,99.99),(10,99.99),(10,99.99),(10,99.99)]
abetaDist=[400,800]

allAbeta_mean={400:[],800:[]}
allAbeta_area={400:[],800:[]}

for path in os.listdir(dataDir):
    print(path)
    for fname in os.listdir(os.path.join(dataDir,path)):
        if '.npy' not in fname:
            continue
        pID=fname[:-4]
        print(pID)
        img2d=np.load(os.path.join(dataDir,path,pID+'.npy')).astype(float)
        imgSeg=io.imread(os.path.join(segsavepath,path,pID+'.tif'))
        
        #scale intensity
        for c in range(img2d.shape[0]):
#             if c!=4:
#                 continue
            img2d[c]=np.clip(img2d[c],np.percentile(img2d,minmaxScale[c][0]),np.percentile(img2d,minmaxScale[c][1]))
            img2d[c]=(img2d[c]-np.min(img2d[c]))/(np.max(img2d[c])-np.min(img2d[c]))
        
        #HC of cells
        currCoord=np.zeros((np.unique(imgSeg).size-1,2))
        currHC_staci=np.zeros(np.unique(imgSeg).size-1)
        currHC_std=np.zeros(np.unique(imgSeg).size-1)
        imgCount=0
        for idx in np.unique(imgSeg):
            if idx==0:
                continue
            current_nuc=imgSeg==idx
            
            x,y = center_of_mass(current_nuc)
            x=int(x)
            y=int(y)
            if x-radius<0 or x+radius>(img2d.shape[1]) or y-radius<0 or y+radius>(img2d.shape[2]):
                continue
            
            if np.sum(current_nuc)<minNucSize:
                lmnbPix=img2d[3][current_nuc]>0
                if np.sum(lmnbPix)/np.sum(current_nuc)<laminThreshold:
                    continue
            currCoord[imgCount]=np.array([x,y])
            
            
            
            imgCount+=1
        currCoord=currCoord[:imgCount]
        currImgName=np.repeat(pID,imgCount)
        currPatientID=np.repeat(pID.split('-')[0]+'-'+pID.split('-')[1],imgCount)
        currCat=np.repeat(path,imgCount)
        print(imgCount)
        if allCoord is None:
            allImgNames=currImgName
            allCoord=currCoord
            allPatientID=currPatientID
            allCat=currCat
        else:
            allImgNames=np.concatenate((allImgNames,currImgName))
            allCoord=np.concatenate((allCoord,currCoord),axis=0)
            allPatientID=np.concatenate((allPatientID,currPatientID))
            allCat=np.concatenate((allCat,currCat))
        
            

with open(os.path.join(savedir_processed,'allImgNames'), 'wb') as output:
    pickle.dump(allImgNames, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allCoord'), 'wb') as output:
    pickle.dump(allCoord, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allPatientID'), 'wb') as output:
    pickle.dump(allPatientID, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allCat'), 'wb') as output:
    pickle.dump(allCat, output, pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(savedir_processed,'allHC_staci'), 'wb') as output:
#     pickle.dump(allHC_staci, output, pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(savedir_processed,'allHC_std'), 'wb') as output:
#     pickle.dump(allHC_std, output, pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(savedir_processed,'allAbeata_mean'), 'wb') as output:
#     pickle.dump(allAbeta_mean, output, pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(savedir_processed,'allAbeata_area'), 'wb') as output:
#     pickle.dump(allAbeta_area, output, pickle.HIGHEST_PROTOCOL)



In [15]:
radius

64

In [10]:
with open(os.path.join(savedir_processed,'allHC_staci'), 'rb') as output:
    allHC_staci=pickle.load(output)
with open(os.path.join(savedir_processed,'allHC_std'), 'rb') as output:
    allHC_std=pickle.load(output)
with open(os.path.join(savedir_processed,'allAbeata_mean'), 'rb') as output:
    allAbeta_mean=pickle.load( output)
with open(os.path.join(savedir_processed,'allAbeata_area'), 'rb') as output:
    allAbeta_area=pickle.load(output)


In [ ]:
allImgs=None

imgSize_target=128
imgSize=imgSize_target*scaleFactor
radius=int(imgSize/2)
minNucSize=300
laminThreshold=0.3 #by area
minmaxScale=[(10,99.99),(90,99.99),(10,99.99),(10,99.99),(10,99.99)]

for path in os.listdir(dataDir):
    print(path)
    for fname in os.listdir(os.path.join(dataDir,path)):
        if '.npy' not in fname:
            continue
        pID=fname[:-4]
        print(pID)
        img2d=np.load(os.path.join(dataDir,path,pID+'.npy')).astype(float)
        imgSeg=io.imread(os.path.join(segsavepath,path,pID+'.tif'))
        
        #scale intensity
        for c in range(img2d.shape[0]):
#             if c!=4:
#                 continue
            img2d[c]=np.clip(img2d[c],np.percentile(img2d,minmaxScale[c][0]),np.percentile(img2d,minmaxScale[c][1]))
            img2d[c]=(img2d[c]-np.min(img2d[c]))/(np.max(img2d[c])-np.min(img2d[c]))
        
        currImgs=np.zeros((np.unique(imgSeg).size-1,5,imgSize_target,imgSize_target))
        imgCount=0
        for idx in np.unique(imgSeg):
            if idx==0:
                continue
            current_nuc=imgSeg==idx
            
            x,y = center_of_mass(current_nuc)
            x=int(x)
            y=int(y)
            if x-radius<0 or x+radius>(img2d.shape[1]) or y-radius<0 or y+radius>(img2d.shape[2]):
                continue
            
            if np.sum(current_nuc)<minNucSize:
                lmnbPix=img2d[3][current_nuc]>0
                if np.sum(lmnbPix)/np.sum(current_nuc)<laminThreshold:
                    continue
            
            
            currImgs_0=np.copy(img2d[4,x-radius:x+radius,y-radius:y+radius])
            currImgs_0_seg=imgSeg[x-radius:x+radius,y-radius:y+radius]
            currImgs_0[currImgs_0_seg!=idx]=0
            
            currImgs_2=np.copy(img2d[3,x-radius:x+radius,y-radius:y+radius])
            currImgs_2[currImgs_0_seg!=idx]=0
            
            currImgs[imgCount,0]=cv2.resize(currImgs_0, (imgSize_target,imgSize_target)) #dapi
            currImgs[imgCount,1]=cv2.resize(img2d[2,x-radius:x+radius,y-radius:y+radius], (imgSize_target,imgSize_target)) #map2
            currImgs[imgCount,3]=cv2.resize(img2d[1,x-radius:x+radius,y-radius:y+radius], (imgSize_target,imgSize_target)) #abeta
            currImgs[imgCount,2]=cv2.resize(currImgs_2, (imgSize_target,imgSize_target)) #lmnb
            currImgs[imgCount,4]=cv2.resize(img2d[0,x-radius:x+radius,y-radius:y+radius], (imgSize_target,imgSize_target)) #tau
            
            imgCount+=1
        currImgs=currImgs[:imgCount]
        if allImgs is None:
            allImgs=currImgs
        else:
            allImgs=np.concatenate((allImgs,currImgs))
        
            

with open(os.path.join(savedir_processed,'allImgs_minmax_segNuc'), 'wb') as output:
    pickle.dump(allImgs, output, pickle.HIGHEST_PROTOCOL)

